In [3]:
!pip install faker
import sqlite3
import random
from faker import Faker

# Initialize Faker for realistic data
fake = Faker()

# Connect to SQLite
conn = sqlite3.connect('garston_superstore.db')
cursor = conn.cursor()

# Step 1: Create Tables
cursor.execute('''
CREATE TABLE IF NOT EXISTS Products (
    Product_ID TEXT PRIMARY KEY,  -- Alphanumeric ID
    Product_Name TEXT NOT NULL,
    Category TEXT NOT NULL,
    Price REAL NOT NULL CHECK(Price > 0)
);
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Employees (
    Employee_ID INTEGER PRIMARY KEY AUTOINCREMENT,
    Name TEXT NOT NULL,
    Role TEXT NOT NULL,
    Hourly_Rate REAL NOT NULL CHECK(Hourly_Rate > 0)
);
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Customers (
    Customer_ID TEXT PRIMARY KEY,  -- Prefixed with "CUST"
    Name TEXT NOT NULL,
    Age_Group TEXT NOT NULL,
    Postcode TEXT
);
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Sales (
    Sale_ID INTEGER PRIMARY KEY AUTOINCREMENT,
    Product_ID TEXT NOT NULL,
    Employee_ID INTEGER NOT NULL,
    Customer_ID TEXT,
    Sale_Date TEXT NOT NULL,
    Quantity INTEGER NOT NULL CHECK(Quantity > 0),
    Total_Price REAL NOT NULL CHECK(Total_Price > 0),
    FOREIGN KEY(Product_ID) REFERENCES Products(Product_ID),
    FOREIGN KEY(Employee_ID) REFERENCES Employees(Employee_ID),
    FOREIGN KEY(Customer_ID) REFERENCES Customers(Customer_ID)
);
''')

# Step 2: Populate Data

# 1. Products Table with unique alphanumeric IDs and realistic product names
categories = {
    'Groceries': ['Rice', 'Pasta', 'Cereal', 'Oil', 'Sugar'],
    'Bakery': ['Bread', 'Croissant', 'Bagel', 'Muffin', 'Cake'],
    'Dairy': ['Milk', 'Cheese', 'Yogurt', 'Butter', 'Cream'],
    'Snacks': ['Chips', 'Cookies', 'Popcorn', 'Chocolate', 'Candy'],
    'Beverages': ['Juice', 'Soda', 'Tea', 'Coffee', 'Water']
}

# Generate 100 unique product IDs
product_ids = [f"AC{str(i).zfill(3)}" for i in range(100)]  # AC001, AC002, etc.

# Create Product entries
products = []
for product_id in product_ids:
    category = random.choice(list(categories.keys()))
    product_name = random.choice(categories[category])
    price = round(random.uniform(0.5, 50.0), 2)
    products.append((product_id, product_name, category, price))

cursor.executemany('INSERT INTO Products (Product_ID, Product_Name, Category, Price) VALUES (?, ?, ?, ?)', products)

# 2. Employees Table
roles = ['Cashier', 'Manager', 'Stocker']
employees = [
    (fake.name(), random.choice(roles), round(random.uniform(10.0, 25.0), 2))
    for _ in range(50)
]
cursor.executemany('INSERT INTO Employees (Name, Role, Hourly_Rate) VALUES (?, ?, ?)', employees)

# 3. Customers Table with CUST prefix
age_groups = ['18-25', '26-35', '36-45', '46-55', '56-65', '66+']
postcodes = [f'L19 {i:02d}' for i in range(1, 21)]
customers = [
    (f"CUST{str(i).zfill(4)}", fake.name(), random.choice(age_groups), random.choice(postcodes) if random.random() > 0.05 else None)
    for i in range(1, 501)
]
cursor.executemany('INSERT INTO Customers (Customer_ID, Name, Age_Group, Postcode) VALUES (?, ?, ?, ?)', customers)

# 4. Sales Table
sales = []
for _ in range(1000):
    product_id = random.choice(product_ids)  # Random product ID
    employee_id = random.randint(1, 50)  # Random Employee ID
    customer_id = f"CUST{random.randint(1, 500):04}" if random.random() > 0.1 else None  # 10% NULL Customer_ID
    quantity = random.randint(1, 10)
    sale_date = fake.date_between(start_date='-30d', end_date='today')
    total_price = round(random.uniform(5.0, 500.0), 2)  # Random total price
    sales.append((product_id, employee_id, customer_id, str(sale_date), quantity, total_price))

cursor.executemany('''
INSERT INTO Sales (Product_ID, Employee_ID, Customer_ID, Sale_Date, Quantity, Total_Price)
VALUES (?, ?, ?, ?, ?, ?)
''', sales)

# Commit changes and close connection
conn.commit()
conn.close()


In [2]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 18.4 MB/s eta 0:00:00
